In [1]:
import os
import pandas as pd
import instructor
import openai
from pydantic import Field, field_validator, BaseModel
from typing import Optional

In [62]:
res_df = pd.read_csv("/Users/shreeyadhakal/Desktop/Projects-2023/llama-3-finetune/infer_out/hindi_lora_all_scores.csv")

In [45]:
res_df.head()

,input,instruction,output,generated_text,model_response,cleaned_response,relevance_score,cc_score,syntax_score,complete_score,hallucination_type,is_repeat,expected_out_word_count,actual_out_word_count,rouge1,rouge2,rougeL,bleurt
0,वह पार्टी से जल्दी चली गई,निम्नलिखित वाक्य को और अधिक दिलचस्प बनाने के ल...,वह पार्टी से जल्दी ही चली गई।,<|begin_of_text|>Below is an instruction in Hi...,"वह पार्टी से जल्दी चली गई, क्योंकि वह काम पर ज...","वह पार्टी से जल्दी चली गई, क्योंकि वह काम पर ज...",8,7,8,7,NaN,No,7,14,0.000000,0.000000,0.000000,0.715245
1,NaN,"f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1....",हम समीकरण f(x) = 0 को x ≤ 1 और x > 1 डोमेन पर ...,<|begin_of_text|>Below is an instruction in Hi...,"f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1....","f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1....",1,1,1,1,nonsensical,yes,99,31,0.545455,0.490566,0.472727,0.469601
2,NaN,एक गर्मियों के दिन के बारे में एक हाइकु कविता ...,मेरे चेहरे पर गर्म हवा अंतहीन सूरज खुशी और शां...,<|begin_of_text|>Below is an instruction in Hi...,काली छत्तीस\nलहरों में बिखरी हुई\nबगीचे की बेल,काली छत्तीस\nलहरों में बिखरी हुई\nबगीचे की बेल...,4,5,6,3,NaN,No,17,10,0.000000,0.000000,0.000000,0.145238
3,NaN,मशीन लर्निंग मॉडल की सटीकता में सुधार के लिए क...,मशीन लर्निंग मॉडल की सटीकता में सुधार करने के ...,<|begin_of_text|>Below is an instruction in Hi...,NaN,<|end_of_text|>,1,1,1,1,NaN,No,187,1,0.000000,0.000000,0.000000,0.314681
4,________ और __________ को कम करके ग्लोबल वार्म...,वाक्य को पूरा करने के लिए रिक्त स्थान भरें।,ग्रीनहाउस गैस उत्सर्जन और वनों की कटाई को कम क...,<|begin_of_text|>Below is an instruction in Hi...,बिजली के बिना और कार्बन डाइऑक्साइड के बिना,बिजली के बिना और कार्बन डाइऑक्साइड के बिना<|en...,6,6,8,5,NaN,No,18,8,0.000000,0.000000,0.000000,0.175979


In [21]:
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

In [22]:
class Score(BaseModel):
    relevance_score: Optional[int] = Field("Score the response for relevance to the given instruction on a scale of 1 (lowest) to 10 (highest).")
    cc_score: Optional[int] = Field("Score the response for clarity and coherence on a scale of 1 (lowest) to 10 (highest).")
    syntax_score: Optional[int] = Field("Score the response for correctness of syntax and grammar on a scale of 1 (lowest) to 10 (highest).")
    complete_score: Optional[int] = Field("Score the response for completeness on a scale of 1 (lowest) to 10 (highest).")
    hallucination_type: Optional[str] = Field("Give the type of hallucination in the model response: factual, nonsensical, contradiction or other. If there is no hallucination say None.")
    is_repeat: Optional[str] = Field("Is the rensponse repeat of the Instruction or Input. Return Yes or No")


In [25]:
# class IsaRepeat(BaseModel):
    

#     @field_validator('is_repeat')
#     def score_validator(cls, s):
#         try:
#             if not isinstance(s, str):
#                 raise ValueError("The value must be an string.")
   
#             elif s.lower() not in ["yes", "no"]:
#                 raise ValueError("The response should be either yes or no.")
#             else:
#                 return s
#         except ValueError as e:
#             print(f"Validation error: {e}")

In [6]:
def get_content(input, ins, model_out):
    return f"""
        Grade the response of the model given the instruction and input.\n
        ### Start of Instruction and Input###

        {ins} + {input}

        ### End of Instruction ###
        ### Start of response ###

        {model_out}
        
        ### End of response ###
        """

In [7]:
os.environ["OPENAI_API_KEY"]=load_api_key("../../.keys")

In [8]:
from openai import OpenAI

client = instructor.from_openai(OpenAI())

In [63]:
from tqdm import tqdm 

scores = []
# res_df_=res_df[:10]
for idx, row in tqdm(res_df.iterrows(), total=len(res_df)):
    try:
        input = row["input"].strip()
    except Exception as e:
        input = ""
    ins = row ["instruction"].strip()
    expected = row["output"].strip()
    model_out = row["cleaned_response"].strip()
    model_out_ = model_out.replace("<|end_of_text|>", "")
    
    res = {}
    if len(model_out_) == 0:
        res["relevance_score"], res["cc_score"], res["syntax_score"], res["complete_score"] = 0, 0, 0, 0
        res["hallucination_type"] = "Empty Response"
        res["is_repeat"] = "Empty Response"
        scores.append(res)
    else:
        try:
            response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                    {"role": "user", "content": get_content(input, ins, model_out)},
                ],
            response_model = Score,
            max_tokens=256,
            temperature=0.7,
            )
            res = response.model_dump()
            scores.append(res)
        except Exception as e:
            print(f"Falied to process row {idx} with exception {e}")

scores_df = pd.DataFrame(scores)

  0%|          | 0/199 [00:00<?, ?it/s]

100%|██████████| 199/199 [03:05<00:00,  1.07it/s]


In [64]:
scores_df

,relevance_score,cc_score,syntax_score,complete_score,hallucination_type,is_repeat
0,8,7,8,7,None,No
1,5,5,5,5,factual,No
2,7,8,9,7,None,No
3,8,5,6,7,None,No
4,9,9,9,9,None,No
...,...,...,...,...,...,...
194,10,10,10,10,None,No
195,8,7,8,9,None,No
196,6,3,4,3,None,Yes
197,9,8,7,8,None,No


In [65]:
scores_df["hallucination_type"].value_counts()

hallucination_type
None                       154
factual                     22
nonsensical                 13
contradiction                4
repeat                       2
factual, nonsensical         1
other                        1
factual and nonsensical      1
Repeat                       1
Name: count, dtype: int64

In [66]:
scores_df.loc[scores_df["hallucination_type"] == "None", "hallucination_type"] = "none"
scores_df.loc[scores_df["hallucination_type"] == "No", "hallucination_type"] = "none"
scores_df.loc[scores_df["hallucination_type"] == "repeat", "hallucination_type"] = "repetition"
scores_df.loc[scores_df["hallucination_type"] == "factual and nonsensical", "hallucination_type"] = "nonsensical"
scores_df.loc[scores_df["hallucination_type"] == "repeat text", "hallucination_type"] = "repetition"
scores_df.loc[scores_df["hallucination_type"] == "factual, nonsensical", "hallucination_type"] = "nonsensical"
scores_df.loc[scores_df["hallucination_type"] == "Repeat", "hallucination_type"] = "repetition"
scores_df["hallucination_type"].value_counts()
# scores_df.loc[df["is_repeat"] == "yes", "is_repeat"] = "Yes"

hallucination_type
none             154
factual           22
nonsensical       15
contradiction      4
repetition         3
other              1
Name: count, dtype: int64

In [67]:
final_df = pd.concat([res_df, scores_df], axis=1)

In [68]:
final_df

,input,instruction,output,generated_text,model_response,cleaned_response,relevance_score,cc_score,syntax_score,complete_score,...,syntax_score.1,complete_score.1,hallucination_type.1,is_repeat.1,relevance_score,cc_score,syntax_score,complete_score,hallucination_type,is_repeat
0,वह पार्टी से जल्दी चली गई,निम्नलिखित वाक्य को और अधिक दिलचस्प बनाने के ल...,वह पार्टी से जल्दी ही चली गई।,<|begin_of_text|>Below is an instruction in Hi...,वह जल्दी पार्टी से चली गई।,वह जल्दी पार्टी से चली गई।<|end_of_text|>,7,6,8,5,...,9,5,none,Yes,8,7,8,7,none,No
1,NaN,"f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1....",हम समीकरण f(x) = 0 को x ≤ 1 और x > 1 डोमेन पर ...,<|begin_of_text|>Below is an instruction in Hi...,"f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1....","f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1....",5,4,4,3,...,5,4,factual,Yes,5,5,5,5,factual,No
2,NaN,एक गर्मियों के दिन के बारे में एक हाइकु कविता ...,मेरे चेहरे पर गर्म हवा अंतहीन सूरज खुशी और शां...,<|begin_of_text|>Below is an instruction in Hi...,"सुनहरी सूर्यास्त, गर्म हवा, गर्मियों का दिन, श...","सुनहरी सूर्यास्त, गर्म हवा, गर्मियों का दिन, श...",8,6,7,5,...,9,7,none,No,7,8,9,7,none,No
3,NaN,मशीन लर्निंग मॉडल की सटीकता में सुधार के लिए क...,मशीन लर्निंग मॉडल की सटीकता में सुधार करने के ...,<|begin_of_text|>Below is an instruction in Hi...,मशीन लर्निंग मॉडल की सटीकता में सुधार करने के ...,मशीन लर्निंग मॉडल की सटीकता में सुधार करने के ...,8,7,6,6,...,7,7,none,No,8,5,6,7,none,No
4,________ और __________ को कम करके ग्लोबल वार्म...,वाक्य को पूरा करने के लिए रिक्त स्थान भरें।,ग्रीनहाउस गैस उत्सर्जन और वनों की कटाई को कम क...,<|begin_of_text|>Below is an instruction in Hi...,कार्बन उत्सर्जन और ग्रीनहाउस गैसों को कम करके ...,कार्बन उत्सर्जन और ग्रीनहाउस गैसों को कम करके ...,10,10,10,10,...,10,10,none,No,9,9,9,9,none,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,NaN,दुनिया का सबसे ऊंचा पर्वत कौन सा है?,"सगरमाथा विश्व का सबसे ऊंचा पर्वत है, जिसकी समु...",<|begin_of_text|>Below is an instruction in Hi...,"विश्व का सबसे ऊंचा पर्वत माउंट एवरेस्ट है, जो ...","विश्व का सबसे ऊंचा पर्वत माउंट एवरेस्ट है, जो ...",10,10,10,10,...,10,10,none,No,10,10,10,10,none,No
195,NaN,परमाणु ऊर्जा के चार फायदे बताइए।,1. दक्षता और विश्वसनीयता: परमाणु ऊर्जा संयंत्र...,<|begin_of_text|>Below is an instruction in Hi...,1. स्थिर ऊर्जा स्रोत: परमाणु ऊर्जा एक स्थिर और...,1. स्थिर ऊर्जा स्रोत: परमाणु ऊर्जा एक स्थिर और...,9,8,8,9,...,8,9,none,No,8,7,8,9,none,No
196,NaN,कोशिका में माइटोकॉन्ड्रिया की भूमिका की व्याख्...,"माइटोकॉन्ड्रिया को अक्सर कोशिका की ""ऊर्जा शक्त...",<|begin_of_text|>Below is an instruction in Hi...,"माइटोकॉन्ड्रिया को कोशिका के भीतर ""ऊर्जा के लि...","माइटोकॉन्ड्रिया को कोशिका के भीतर ""ऊर्जा के लि...",5,2,3,3,...,3,3,none,Yes,6,3,4,3,none,Yes
197,NaN,मनगढ़ंत लेखन में इस्तेमाल की जाने वाली चार तकन...,1. भावनात्मक अपील: प्रेरक लेखक अक्सर पाठक की भ...,<|begin_of_text|>Below is an instruction in Hi...,1. प्रेरक भाषा: मनगढ़ंत लेखन में अक्सर प्रेरक ...,1. प्रेरक भाषा: मनगढ़ंत लेखन में अक्सर प्रेरक ...,7,6,6,7,...,8,8,none,No,9,8,7,8,none,No


In [69]:
final_df.to_csv("../../infer_out/hindi_lora_all_scores.csv", index=False)